In [1]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
 
#download the treebank corpus from nltk
nltk.download('treebank')
 
#download the universal tagset from nltk
nltk.download('universal_tagset')
 
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
 
#print the first two sentences along with tags
print(nltk_data[:2])

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [2]:
#print each word with its respective tag for first two sentences
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


In [5]:
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [19]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))
print(test_set[:5])

80310
20366
[[('The', 'DET'), ('company', 'NOUN'), ('said', 'VERB'), ('0', 'X'), ('it', 'PRON'), ('is', 'VERB'), ('in', 'ADP'), ('the', 'DET'), ('process', 'NOUN'), ('of', 'ADP'), ('*', 'X'), ('phasing', 'VERB'), ('out', 'PRT'), ('John', 'NOUN'), ('Deere', 'NOUN'), (',', '.'), ('its', 'PRON'), ('current', 'ADJ'), ('source', 'NOUN'), ('of', 'ADP'), ('production', 'NOUN'), ('for', 'ADP'), ('midsized', 'ADJ'), ('motor', 'NOUN'), ('home', 'NOUN'), ('chassis', 'NOUN'), ('.', '.')], [('Douglas', 'NOUN'), ('Madison', 'NOUN'), (',', '.'), ('a', 'DET'), ('corporate', 'ADJ'), ('trader', 'NOUN'), ('with', 'ADP'), ('Bank', 'NOUN'), ('of', 'ADP'), ('America', 'NOUN'), ('in', 'ADP'), ('Los', 'NOUN'), ('Angeles', 'NOUN'), (',', '.'), ('traced', 'VERB'), ('the', 'DET'), ('dollar', 'NOUN'), ("'s", 'PRT'), ('recent', 'ADJ'), ('solid', 'ADJ'), ('performance', 'NOUN'), ('against', 'ADP'), ('the', 'DET'), ('yen', 'NOUN'), ('to', 'PRT'), ('purchases', 'NOUN'), ('of', 'ADP'), ('securities', 'NOUN'), ('by', '

In [7]:
# check some of the tagged words.
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

In [8]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'ADJ', '.', 'ADV', 'NOUN', 'X', 'DET', 'CONJ', 'PRT', 'VERB', 'ADP', 'NUM', 'PRON'}


In [9]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [10]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [11]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[6.33009672e-02 6.60194159e-02 5.24271838e-03 6.96893215e-01
  2.09708735e-02 5.24271838e-03 1.68932043e-02 1.14563107e-02
  1.14563107e-02 8.05825219e-02 2.17475723e-02 1.94174761e-04]
 [4.61323895e-02 9.23720598e-02 5.25694676e-02 2.18538776e-01
  2.56410260e-02 1.72191828e-01 6.00793920e-02 2.78940029e-03
  8.96899477e-02 9.29084867e-02 7.82104954e-02 6.87694475e-02]
 [1.30721495e-01 1.39255241e-01 8.14584941e-02 3.21955010e-02
  2.28859577e-02 7.13731572e-02 6.98215654e-03 1.47401085e-02
  3.39022487e-01 1.19472459e-01 2.98681147e-02 1.20248254e-02]
 [1.25838192e-02 2.40094051e-01 1.68945398e-02 2.62344331e-01
  2.88252197e-02 1.31063312e-02 4.24540639e-02 4.39345129e-02
  1.49133503e-01 1.76826611e-01 9.14395228e-03 4.65906132e-03]
 [1.76821072e-02 1.60868734e-01 2.57543717e-02 6.16951771e-02
  7.57255405e-02 5.68902567e-02 1.03786280e-02 1.85085520e-01
  2.06419379e-01 1.42225638e-01 3.07514891e-03 5.41995019e-02]
 [2.06410810e-01 1.73925534e-02 1.20741697e-02 6.35906279e-01
  4

In [12]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,ADJ,.,ADV,NOUN,X,DET,CONJ,PRT,VERB,ADP,NUM,PRON
ADJ,0.063301,0.066019,0.005243,0.696893,0.020971,0.005243,0.016893,0.011456,0.011456,0.080583,0.021748,0.000194
.,0.046132,0.092372,0.052569,0.218539,0.025641,0.172192,0.060079,0.002789,0.089690,0.092908,0.078210,0.068769
ADV,0.130721,0.139255,0.081458,0.032196,0.022886,0.071373,0.006982,0.014740,0.339022,0.119472,0.029868,0.012025
NOUN,0.012584,0.240094,0.016895,0.262344,0.028825,0.013106,0.042454,0.043935,0.149134,0.176827,0.009144,0.004659
X,0.017682,0.160869,0.025754,0.061695,0.075726,0.056890,0.010379,0.185086,0.206419,0.142226,0.003075,0.054200
DET,0.206411,0.017393,0.012074,0.635906,0.045134,0.006037,0.000431,0.000287,0.040247,0.009918,0.022855,0.003306
CONJ,0.113611,0.035126,0.057080,0.349067,0.009330,0.123491,0.000549,0.004391,0.150384,0.055982,0.040615,0.060373
PRT,0.082975,0.045010,0.009393,0.250489,0.012133,0.101370,0.002348,0.001174,0.401174,0.019569,0.056751,0.017613
VERB,0.066390,0.034807,0.083886,0.110589,0.215930,0.133610,0.005433,0.030663,0.167956,0.092357,0.022836,0.035543
ADP,0.107062,0.038724,0.014553,0.323589,0.034548,0.320931,0.001012,0.001266,0.008479,0.016958,0.063275,0.069603


In [13]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [20]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
print(test_run)
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
print(test_run_base)
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
print(test_tagged_words)

[[('The', 'DET'), ('company', 'NOUN'), ('is', 'VERB'), ('contesting', 'VERB'), ('the', 'DET'), ('fine', 'NOUN'), ('.', '.')], [('``', '.'), ('I', 'PRON'), ("'m", 'VERB'), ('starting', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('see', 'VERB'), ('more', 'ADJ'), ('business', 'NOUN'), ('transactions', 'NOUN'), (',', '.'), ("''", '.'), ('says', 'VERB'), ('*T*-2', 'X'), ('Andrea', 'NOUN'), ('West', 'NOUN'), ('of', 'ADP'), ('American', 'NOUN'), ('Telephone', 'NOUN'), ('&', 'CONJ'), ('Telegraph', 'NOUN'), ('Co.', 'NOUN'), (',', '.'), ('*-3', 'X'), ('noting', 'VERB'), ('growing', 'VERB'), ('interest', 'NOUN'), ('in', 'ADP'), ('use', 'NOUN'), ('of', 'ADP'), ('900', 'NUM'), ('service', 'NOUN'), ('for', 'ADP'), ('stock', 'NOUN'), ('sales', 'NOUN'), (',', '.'), ('software', 'NOUN'), ('tutorials', 'NOUN'), ('and', 'CONJ'), ('even', 'ADV'), ('service', 'NOUN'), ('contracts', 'NOUN'), ('.', '.')], [('He', 'PRON'), ('added', 'VERB'), (',', '.'), ('``', '.'), ('This', 'DET'), ('has', 'VERB'), ('nothing', 'N

In [15]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  28.915680408477783
Viterbi Algorithm Accuracy:  93.77990430622009
